In [1]:
##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import torch
import sklearn

In [3]:
# seed 고정
torch.manual_seed(1)

In [4]:
# data load
train = pd.read_csv("/kaggle/input/2021-ai-quiz1-p2/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-quiz1-p2/test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-quiz1-p2/submit_sample.csv")

In [5]:
# data 확인
print(train.head())
print(test.head())
print(submit.head())

In [6]:
# data 전처리 및 텐서에 올리기
# 데이터 전처리 를 위해(scale 조정)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train = np.array(train.drop(['0','8'],axis =1 ))
y_train = np.array(train['8'])
x_test = np.array(test.drop(['0','8'],axis=1))

scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

x_train = torch.Tensor(x_train).cuda()
x_test = torch.Tensor(x_test).cuda()
y_train= torch.Tensor(y_train).unsqueeze(1).cuda()

In [7]:
# data 모양 확인
print(x_train.shape,y_train.shape)

In [8]:
# 학습 세팅
w = torch.zeros((8,1),requires_grad=True,device="cuda")
b = torch.zeros(1,requires_grad=True,device="cuda")

epochs = 1000
optim = torch.optim.SGD([w,b],lr= 1e-2)
loss = torch.nn.BCELoss()

In [9]:
# 학습 진행
for epoch in range(epochs+1):
    optim.zero_grad()
    
    h=torch.sigmoid(x_train.matmul(w)+b)
    cost = loss(h,y_train)
    
    cost.backward()
    optim.step()
    
    if epoch % 100 == 0:
        print(cost.item())

In [10]:
# 데이터 예측
with torch.no_grad():
    predict = torch.sigmoid(x_test.matmul(w)+b)
# sigmoid를 통과한 예측값이 .5  보다 크면 1, 작으면 0을 위해 T/F로 변환
predict = predict >= torch.Tensor([0.5]).cuda()
# T/F를 int 변환 T->1 F->0
predict = predict.int()


In [11]:
submit['Label'] = predict.cpu().numpy()
submit.to_csv("submission.csv",index=False)


# try
# lr 1e-4 : 0.6 부근에서 cost가 줄어들지 않음
# lr 1e-3 : cost가 위의 경우보다는 감소하였으나 계속 감소할 것 같음
# lr 1e-2 : cost가 나름 감소를 멈춰감 -> epoch를 5000으로 늘려보다 0.48쯤에 고정됨